Getting those olders amount of data, 'bout confirmed and suspects cases.

> In this script we just want to know how much suspects and confirmed cases are in the mondays, after getting result, we push to a csv file that will be used later in the main program, to generate a new plot.

In [1]:
from oauth2client.service_account import ServiceAccountCredentials as Credentials
import gspread # Sheets

# Login Constants
SCOPE = [
         'https://spreadsheets.google.com/feeds', 
         'https://www.googleapis.com/auth/drive'
]
URL = 'https://docs.google.com/spreadsheets/d/1YERxbIO0VgnVLElEgFxsv_Hr2ra6nHuPVUUGx6OVds4/edit?usp=sharing'

# Create an Client
GAUTH = Credentials.from_json_keyfile_name('../credentials/nisis_credentials.json',SCOPE)
# Auth the client
GCLIENT = gspread.authorize(GAUTH)

In [2]:
import pandas as pd
from numpy import nan as NaN
import numpy as np

def getSheetsNames(URL, gc):
  '''
  Get all tab's names from url (sheet)
  
  Parameters
  ----------
  URL: (str)
    Url's sheet
  gc: (GoogleClient)
    Google client from gspread autenticated.

  Return
  ------
    (list) Sheet's tab names.
  '''
  workSheets = GCLIENT.open_by_url(URL)
  sheetsNames = [i.title for i in workSheets.worksheets()] # sheets names
  return sheetsNames


def getSheetValue(sheet_name, URL, gc, debug=False):
  '''
  Get pandas DF from sheet_name in WorkSheets. 
  
  Parameters
  ----------
  sheet_name: (string)
    Name of sheet

  Return
  ------
  (DataFrame) 2Dimensional sheet got it from one tab.

  Examples
  --------
    >> all_dfs = list(map(getSheetValue,sheets))
    >> list(map(lambda x: x.columns, all_dfs) )
    >> df = pd.concat(all_dfs)
  '''
  wb    = GCLIENT.open_by_url(URL) # client open this url
  sheet = wb.worksheet(sheet_name) # get value(tab) settled from cell above
  data  = sheet.get_all_values() # get csv content from spreadsheet
  df    = pd.DataFrame(data) # convert csv into DataFrame 

  df.columns = df.iloc[0] # Remove Id's columns (They're in fst row)
  df = df.iloc[1:] # Ignore fst row
  
  # If we have an empty column
  df.dropna(axis='columns')
  # If we have an empty row
  df.dropna(axis='rows')
  # Drop fst column (Index)
  df.drop(df.columns[0], axis=1, inplace=True)


  ############################  Treating data ##################################
  # Renaming columns
  #   We rename column by column, 'cause if we do in this way, we won't have
  #   problems with insertion of new columns. If we made this by replacing columns
  #   list to a new one.
  df.rename(columns={'Selecione a situação:': "Situation"}, inplace=True)
  # Remove where Situation is empty -- In this dataframe, empty means that are duplicate
  _filter = df['Situation'] != ''
  df = df[_filter]
  
   # Convert to lower. We do this to minimize possible errors when making a string compare.
  df['Situation'] = df['Situation'].str.lower()
  
  df = df.reset_index(drop=True) # Drop removes old indexation 

  return df

In [ ]:
# Getting results
sheetName = "{}-{}-{}".format(DAY,MONTH,YEAR)
df = getSheetValue(sheetName, URL, GCLIENT)

log.info('Tab "{}" oppened with success!'.format(sheetName))